In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tf_utils import delete_summary_dir, fc_layer, compute_cost, save_tensorboard
from alpha import load_data, iterate_minibatches
from tqdm import tqdm as tqdm #For professional progress-bar
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.framework.graph_util_impl import convert_variables_to_constants
import datetime, time
from datetime import datetime
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [2]:
inception_pb = './inception_pb/inception.pb'
summary_dir = './summary_log'
bottleneck_file = './bottleneck/Bottle_neck_values.csv'

In [3]:
#IMG_WIDTH, IMG_HEIGHT = [299,299]
N_CHANNELS = 3
N_CLASSES = 2
lr = 0.005

In [4]:
#Get the images
TRAIN_DIR = "./train"
IMG_SIZE = 299 #If changing, change in alpha.py as well
img_shape = (IMG_SIZE,IMG_SIZE,3)
num = 300
train, label= load_data(num, IMG_SIZE, DIR =TRAIN_DIR)
label_df = pd.DataFrame(label, columns = ['classes'])
label_unique = label_df.classes.unique()
en = LabelEncoder()
en.fit(label_df.classes)
encoded_label = en.transform(label_df.classes)
label_df["encoded_label"] = encoded_label
target = (np.arange(2) == encoded_label[:,None]).astype(np.float32)

301it [00:00, 363.97it/s]
301it [00:00, 357.80it/s]


In [5]:
train.shape, target.shape

((600, 299, 299, 3), (600, 2))

In [6]:
#To clear summary directory
delete_summary_dir(summary_dir)

In [7]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="inception")
    return graph

In [8]:
def create_bottleneck():
    with tf.Session(graph=graph) as sess:
        im_input = graph.get_tensor_by_name('inception/ExpandDims:0')
        Bottle_neck_values = []
        m = train.shape[0]
        indices = np.arange(m)
        for index in tqdm(range(0, m, 1)): # 1 is when SGD
            batch = indices[index:index + 1]
            Bottle_neck_values.append(sess.run(bottleneck, feed_dict= {im_input:train[batch]}))
        df = pd.DataFrame(np.array(np.squeeze(Bottle_neck_values)))
        df.to_csv("./bottleneck/Bottle_neck_values.csv", index=False)
    return Bottle_neck_values

In [19]:
graph = load_graph(inception_pb)

In [20]:
#To view tensor names
#for op in graph.get_operations():
     #print(op.name)

In [21]:
with graph.as_default():
    with tf.variable_scope("transfer"):
        bottleneck = graph.get_tensor_by_name('inception/pool_3:0')#From inception graph
        Y = tf.placeholder(tf.float32, [None, N_CLASSES])
        X = tf.placeholder(tf.float32, [None, 2048])
        #X = bottleneck
        #fc0 = tf.reshape(X,[-1,2048]) #squeeze
        fc1 = fc_layer(X,   2048, 512,  "fc1", activation = 'relu')
        fc2 = fc_layer(fc1, 512,  128,  "fc2", activation = 'relu')
        logits = fc_layer(fc2,   128,  2,     "logits",       activation =  None)
        cost   = compute_cost(logits,Y)
        var = tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope="transfer/") #trainable var_scope
        optimizer = tf.train.AdamOptimizer(learning_rate=lr, name="adam").minimize(cost, var_list=var) #var_list
        
        with tf.name_scope("accuracy"):
            correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            tf.summary.scalar("accuracy", accuracy)

In [22]:
with tf.Session(graph=graph) as sess:
    filename = "./summary_log/VS-"+time.strftime("%H%M%S")
    writer = tf.summary.FileWriter(filename, sess.graph)
    print("Tensorboard summary saved to "+filename) 

Tensorboard summary saved to ./summary_log/VS-124653


In [23]:
with tf.Session(graph=graph) as sess:
    summaryMerged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(filename, sess.graph)
    #sess.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='transfer/'))
    saver.restore(sess, './saved_model/model')
    
    if not tf.gfile.Exists(bottleneck_file):
        _ = create_bottleneck()
        print("bottleneck file created")
    
    if tf.gfile.Exists(bottleneck_file):
        bottle_neck = np.array(pd.read_csv("./bottleneck/Bottle_neck_values.csv"))
    minibatch_size = 2
    num_epochs = 5
    for epoch in range(num_epochs):
        avg_cost = 0
        for i, minibatch in enumerate(iterate_minibatches(bottle_neck, target, minibatch_size)):
            (minibatch_X, minibatch_Y) = minibatch
            sess.run(optimizer, feed_dict={X: minibatch_X, Y: minibatch_Y})
            avg_cost += sess.run(cost, feed_dict={X: minibatch_X, Y: minibatch_Y})
        summ = sess.run(summaryMerged, feed_dict={X: minibatch_X, Y: minibatch_Y})
        writer.add_summary(summ, i)
        print ("Cost after iteration %i: %f " % (i, avg_cost/(i+1)))
        
        
    print("Training completed!!")
    
    train_accuracy = accuracy.eval({X: bottle_neck, Y: target})
    print("Train Accuracy:", train_accuracy)
    

    #Save weights
    saver = tf.train.Saver()
    saver.save(sess, "./saved_model/model")

INFO:tensorflow:Restoring parameters from ./saved_model/model
Cost after iteration 299: 0.000000 
Cost after iteration 299: 0.000000 
Cost after iteration 299: 0.000000 
Cost after iteration 299: 0.000000 
Cost after iteration 299: 0.000000 
Training completed!!
Train Accuracy: 1.0


In [26]:
graph2 = load_graph(inception_pb)

In [27]:
with graph2.as_default():
    with tf.variable_scope("transfer"):
        bottleneck = graph2.get_tensor_by_name('inception/pool_3:0')#From inception graph
        #Y = tf.placeholder(tf.float32, [None, N_CLASSES])
        #X = tf.placeholder(tf.float32, [None, 2048])
        #X = bottleneck
        fc0 = tf.reshape(bottleneck,[-1,2048]) #squeeze
        fc1 = fc_layer(fc0,   2048, 512,  "fc1", activation = 'relu')
        fc2 = fc_layer(fc1, 512,  128,  "fc2", activation = 'relu')
        logits = fc_layer(fc2,   128,  2,     "logits",       activation =  None)
        prob = tf.nn.softmax(logits, name="prob")
        prediction = tf.argmax(logits, 1, name="prediction")

In [29]:
with tf.Session(graph=graph2) as sess:
    filename = "./summary_log/VS-"+time.strftime("%H%M%S")
    writer = tf.summary.FileWriter(filename, sess.graph)
print("Tensorboard summary saved to "+filename) 

Tensorboard summary saved to ./summary_log/VS-125235


In [ ]:
#for op in graph2.get_operations():
     #print(op.name)

In [30]:
with tf.Session(graph=graph2) as sess:
    saver = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='transfer/'))
    saver.restore(sess, './saved_model/model')
    
    saver2 = tf.train.Saver()
    saver2.save(sess, "./full_model/model")       

INFO:tensorflow:Restoring parameters from ./saved_model/model


In [43]:
graph3 = tf.Graph()
with tf.Session(graph=graph3) as sess:
    saver = tf.train.import_meta_graph('./full_model/model.meta')
    input_graph_def = tf.get_default_graph().as_graph_def()
    saver.restore(sess, "./full_model/model")
    
    
    output_node_names="transfer/prob,transfer/prediction"
    output_graph_def = convert_variables_to_constants(sess,
                                                      input_graph_def,
                                                      output_node_names.split(","))
    

    output_graph="./full_first_try.pb"
    print("Writing to proto buff file")
    with tf.gfile.GFile(output_graph, "wb") as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from ./full_model/model
INFO:tensorflow:Froze 6 variables.
Converted 6 variables to const ops.
Writing to proto buff file
